In [8]:
#now we will download the borzoi data

''''
gsutil -u hopeful-expanse-428320-s8 -m cp \
  "gs://borzoi-paper/data/hg38/tfrecords/fold1-0.tfr" \
  .
'''

#now let's load it in!!

#let's try just loading the TF data with a mapping
import os
import json
import functools
import tensorflow as tf
import numpy as np

path = '/data/leslie/sarthak/data/enformer/data/borzoi/'

def get_dataset(organism, subset, num_threads=8):
    metadata = get_metadata(organism)
    dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                      compression_type='ZLIB',
                                      num_parallel_reads=num_threads)
    dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                          num_parallel_calls=num_threads)
    return dataset

def get_metadata(organism):
    path = '/data/leslie/sarthak/data/enformer/data/borzoi/statistics.json'
    with tf.io.gfile.GFile(path, 'r') as f:
        return json.load(f)

def tfrecord_files(organism, subset):
    return sorted(tf.io.gfile.glob(os.path.join(
        path, f'{subset}-*.tfr'
    )), key=lambda x: int(x.split('-')[-1].split('.')[0]))

def deserialize(serialized_example, metadata):
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
    sequence = tf.cast(sequence, tf.float32)

    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (metadata['target_length'], metadata['num_targets']))
    target = tf.cast(target, tf.float32)

    return {'sequence': sequence, 'target': target}

def get_targets(organism):
    targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
    return pd.read_csv(targets_txt, sep='\t')


subset='train'
# Count the number of examples
dataset = get_dataset('what', 'fold1', num_threads=1)
num_examples = 34021
# print(f'Total number of examples: {num_examples}')

# Access the 2nd example (index 1)
# example_index = 1
# example = get_example_by_index(dataset, organism, subset, example_index)
# print(f'Example at index {example_index}: {example}')

In [9]:
#now batch dataset
dataset

<ParallelMapDataset shapes: {sequence: (524288, 4), target: (6144, 7611)}, types: {sequence: tf.float32, target: tf.float32}>

In [10]:
it = iter(dataset)
print(next(it))

InvalidArgumentError: Input to reshape is a tensor with 524288 values, but the requested shape has 2097152
	 [[{{node Reshape}}]] [Op:IteratorGetNext]

In [11]:
nparray = np.load('/data/leslie/sarthak/data/enformer/data/borzoi/mseqs_unmap.npy')
print(nparray.shape)

(55497, 6144)


In [12]:
nparray

#unsure what this actually is...

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False,  True],
       [ True,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

# tessting the preprocessing of Borzoi